<a href="https://colab.research.google.com/github/Taweilo/theLook_ecommerce/blob/main/theLook_SQL_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All table & columns in the dataset

In [ ]:
-- Check all table with their cols

SELECT
    table_name,
    STRING_AGG((column_name), ', ') AS columns
FROM
    `bigquery-public-data.thelook_ecommerce`.INFORMATION_SCHEMA.COLUMNS
GROUP BY
    table_name;

Create CSV with complete data

In [ ]:
-- All data

SELECT
    oi.id AS order_item_id,
    o.order_id,
    p.category,
    p.name AS product_name,
    brand,
    o.created_at,
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) AS year, -- create year
    EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) AS month, -- create month
    o.status,
    o.shipped_at,
    o.delivered_at,
    o.returned_at,
    oi.sale_price,
    p.cost,
    oi.user_id,
    u.gender,
    u.state,
    u.country,
    u.longitude AS delivery_longitude,
    u.latitude AS delivery_latitude,
    dc.name AS warehouse_name,
    dc.longitude AS warehouse_longitude,
    dc.latitude AS warehouse_latitude,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi-- combine data from order_items
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u -- receive user profile
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p -- receive product info
    ON oi.product_id = p.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii -- receive inventory info
    ON oi.inventory_item_id = ii.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc -- receive distribution info
    ON ii.product_distribution_center_id = dc.id
ORDER BY
    o.created_at DESC;

1. Overall sales performance

In [ ]:
-- order status in 2024 Jan

SELECT
    COUNT(DISTINCT o.order_id) AS total_order,
    SUM(CASE WHEN status ='Complete' THEN 1 ELSE 0 END) AS total_complete_order,
    SUM(CASE WHEN status ='Processing' THEN 1 ELSE 0 END) AS total_processing_order,
    SUM(CASE WHEN status ='Shipped' THEN 1 ELSE 0 END) AS total_shipped_order,
    SUM(CASE WHEN status ='Returned' THEN 1 ELSE 0 END) AS total_returned_order,
    SUM(CASE WHEN status ='Cancelled' THEN 1 ELSE 0 END) AS total_canceled_order,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1;

In [ ]:
-- Sales KPI (Rev, Cost, gross prpfits) of theLook ecommerce

SELECT
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p -- to get cost info
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND oi.status NOT IN ('Cancelled', 'Returned');

In [ ]:
-- theLook sales in different countries

SELECT
    country,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN (
    SELECT
        id, order_id, user_id, product_id, sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND status NOT IN ('Cancelled', 'Returned')
GROUP BY
    country
ORDER BY
    total_revenue DESC;

2. Sales trend

In [ ]:
 -- theLook sales KPI trend

SELECT
    year,
    month,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    (
        SELECT *,
            EXTRACT(YEAR FROM TIMESTAMP(created_at)) AS year,
            EXTRACT(MONTH FROM TIMESTAMP(created_at)) AS month
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
    ) AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
     (
         ( EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2023
      AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) IN (1,2,3,4,5,6,7,8,9,10,11,12))
  OR (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
            AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
    )
    AND status NOT IN ('Cancelled', 'Returned')
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

In [ ]:
-- 環比
SELECT
    year,
    month,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    (
        SELECT *,
            EXTRACT(YEAR FROM TIMESTAMP(created_at)) AS year,
            EXTRACT(MONTH FROM TIMESTAMP(created_at)) AS month
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
    ) AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    (
        (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) IN (2022, 2023)
        AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 12)
        OR (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) IN (2023, 2024)
        AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
    )
    AND o.status NOT IN ('Cancelled', 'Returned')
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

In [ ]:
-- 同比

SELECT
    year,
    month,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    (
        SELECT *,
            EXTRACT(YEAR FROM TIMESTAMP(created_at)) AS year,
            EXTRACT(MONTH FROM TIMESTAMP(created_at)) AS month
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
    ) AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    (
        (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2022
        AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
        OR (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2023
        AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
        OR (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
    )
    AND o.status NOT IN ('Cancelled', 'Returned')
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

3. product performance
- The most popular brand, product, category
- The most cancelled product, category
- The most returned product, category

In [ ]:
-- The most sales category/ Sales KPI (Rev, Cost, gross prpfits) of theLook ecommerce

SELECT
    category,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND oi.status NOT IN ('Cancelled', 'Returned')
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 20;

In [ ]:
-- The most popular brand/ Sales KPI (Rev, Cost, gross prpfits) of theLook ecommerce

SELECT
    p.brand,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND oi.status NOT IN ('Cancelled', 'Returned')
GROUP BY p.brand
ORDER BY total_revenue DESC
LIMIT 20;

In [ ]:
-- The most popular product/ Sales KPI (Rev, Cost, gross prpfits) of theLook ecommerce

SELECT
    p.name,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit,
    COUNT(oi.id) AS unit_sold,
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND oi.status NOT IN ('Cancelled', 'Returned')
GROUP BY p.name
ORDER BY total_revenue DESC
LIMIT 20;

In [ ]:
-- top 20 cancelled brand

SELECT
    brand,
    COUNT(oi.product_id) AS cancelled_frequency,
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Cancelled'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    brand
ORDER BY
    cancelled_frequency DESC
LIMIT 20;

In [ ]:
-- top 20 cancelled product

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS cancelled_frequency,
    ROUND(SUM(oi.sale_price),2) AS possible_refund,
    p.brand,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Cancelled'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, category, p.brand
ORDER BY
    COUNT(oi.product_id) DESC
LIMIT 20;

In [ ]:
-- top 20 returned brand

SELECT
    brand,
    COUNT(oi.product_id) AS returned_frequency,
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Returned'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    brand
ORDER BY
    returned_frequency DESC
LIMIT 20;

In [ ]:
-- top 20 returned product

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS returned_frequency,
    ROUND(SUM(oi.sale_price),2) AS refund,
    p.brand,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Returned'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, category, p.brand
ORDER BY
    returned_frequency
LIMIT 20;

4. Customer profile
- customer location
- age
- gender
- average spend
CLV, RMF?

In [ ]:
-- customers profile from which countries, age, gender

SELECT
    oi.order_id,
    u.id AS user_id,
    u.age,
    u.gender,
    u.country
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;


In [ ]:
-- age profile of the customer

SELECT
    age,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    age
ORDER BY
    age;


In [ ]:
-- age group profile of the customer

SELECT
    age_group,
    COUNT(user_id) AS customer_count
FROM
(
SELECT
        user_id,
        age,
        CASE
            WHEN age BETWEEN 13 AND 19 THEN 'Teens'
            WHEN age BETWEEN 20 AND 35 THEN 'Young Adults'
            WHEN age BETWEEN 36 AND 50 THEN 'Adults'
            ELSE 'Seniors'
        END AS age_group
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
) AS grouped_customers
GROUP BY
    age_group
ORDER BY
    CASE age_group
        WHEN 'Teens' THEN 1
        WHEN 'Young Adults' THEN 2
        WHEN 'Adults' THEN 3
        WHEN 'Seniors' THEN 4
        ELSE 5  -- To handle any unexpected age groups
    END;

In [ ]:
-- country profile of the customer

SELECT
    country,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    country
ORDER BY
    COUNT(DISTINCT u.id) DESC;


In [ ]:
-- gender profile of the customer

SELECT
    gender,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    gender
ORDER BY
    gender;


In [ ]:
-- study the sales price with sales volume

SELECT
    ROUND(oi.sale_price, 2) AS price,
    COUNT(oi.id) AS sales_volume
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    oi.sale_price
ORDER BY
    oi.sale_price;


In [ ]:
-- AOV, APF, Customer value

SELECT
    ROUND((SUM(sale_price) / COUNT(DISTINCT o.order_id)), 2) AS aov, -- Revenue/#Order
    ROUND((COUNT(DISTINCT o.order_id) / COUNT(DISTINCT u.id)), 2) AS apf, -- #Order/Unique users
    ROUND((SUM(sale_price) / COUNT(DISTINCT u.id)), 2) AS customer_value --aov*apf
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.orders` AS o
ON
    oi.order_id = o.order_id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
WHERE
    oi.status NOT IN ('Cancelled', 'Returned')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

In [ ]:
-- Customer buying behavior

WITH rfm_data AS (
    SELECT
        oi.user_id,
        TIMESTAMP_DIFF('2024-01-31', MAX(oi.created_at), DAY) AS recency,
        COUNT(oi.id) AS frequency,
        SUM(oi.sale_price) AS monetary
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
    ON
        oi.product_id = p.id
    WHERE
        oi.status = 'Complete'
        AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
    GROUP BY oi.user_id
)

SELECT
    ROUND(AVG(recency),2) AS Avg_recency,
    ROUND(AVG(frequency),2) AS Avg_frequency,
    ROUND(AVG(monetary),2) AS Avg_monetary

FROM
    rfm_data;


SyntaxError: invalid syntax (<ipython-input-2-4e31a39b56c2>, line 1)

In [ ]:
-- rfm analysis (2019~2024)

WITH rfm_data AS (
    SELECT
        oi.user_id,
        TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), MAX(oi.created_at), DAY) AS recency,
        COUNT(oi.id) AS frequency,
        SUM(oi.sale_price) AS monetary
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
    ON
        oi.product_id = p.id
    WHERE
        oi.status = 'Complete'
        AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2019,2020,2021,2022,2023,2024)
    GROUP BY oi.user_id
)

SELECT
    user_id,
    recency,
    frequency,
    monetary,
    CASE
        WHEN recency <= 30 AND frequency >= 4 AND monetary >= 300 THEN 'Champions'
        WHEN recency <= 30 AND frequency >= 3 THEN 'Loyal Customers'
        WHEN recency <= 60 AND frequency >= 2 THEN 'Potential Loyalists'
        WHEN recency <= 90 THEN 'Recent Customers'
        ELSE 'Needs Attention'
    END AS rfm_segment
FROM
    rfm_data
ORDER BY frequency DESC;

5. Delivery efficency
- Lead time
- Delivery Duration

In [ ]:
-- Overall Avg. lead time/ shipping duration

SELECT
    EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) AS year
    EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) AS month
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR))/24), 2) AS avg_lead_time
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.shipped_at, HOUR))/24), 2) AS avg_shipping_duration
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status IN ('Complete', 'Returned')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2022,2023,2024)
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) IN (1,12);
GROUP BY
    year, month

-- US Avg. lead time/ shipping duration

SELECT
    EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) AS year,
    EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) AS month,
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR))/24), 2) AS avg_lead_time,
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.shipped_at, HOUR))/24), 2) AS avg_shipping_duration
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    ( SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.users`
    WHERE country = 'United States')AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Complete'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2022,2023,2024)
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) IN (1,12)
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

-- Oversea Avg. lead time/ shipping duration
SELECT
    EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) AS year,
    EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) AS month,
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR))/24), 2) AS avg_lead_time,
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.shipped_at, HOUR))/24), 2) AS avg_shipping_duration
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    ( SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.users`
    WHERE country != 'United States')AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Complete'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2022,2023,2024)
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) IN (1,12)
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

6. Marketing approach evaluation
- how many new customer at theLook / through what channel
- did they make the purchase / did they purchase adidas product

In [ ]:
-- total number of the new customers
SELECT
    COUNT(id) AS new_user_num,
FROM
    `bigquery-public-data.thelook_ecommerce.users`
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1;


-- new customer at theLook

SELECT
    id AS user_id,
    traffic_source,
    country,
    age,
    gender
FROM
    `bigquery-public-data.thelook_ecommerce.users`
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

In [ ]:
-- channel traffic

SELECT
    traffic_source,
    COUNT(traffic_source) AS channel_traffic
FROM
    `bigquery-public-data.thelook_ecommerce.users`
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1
GROUP BY
    traffic_source
ORDER BY
    channel_traffic DESC;


SyntaxError: invalid syntax (<ipython-input-1-2e26ff663211>, line 1)

Side proejct 01 RFM analysis
https://clevertap.com/blog/rfm-analysis/

In [ ]:
-- rfm analysis (2019~2024)

SELECT
      oi.user_id,
      TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), MAX(oi.created_at), DAY) AS recency,
      COUNT(oi.id) AS frequency,
      SUM(oi.sale_price) AS monetary
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
ON
  oi.product_id = p.id
WHERE
    oi.status = 'Complete'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2019,2020,2021,2022,2023,2024)
GROUP BY oi.user_id;


In [1]:
-- rfm analysis (2024 Jan)
SELECT
      oi.user_id,
      TIMESTAMP_DIFF('2024-01-31'), MAX(oi.created_at), DAY) AS recency,
      COUNT(oi.id) AS frequency,
      SUM(oi.sale_price) AS monetary
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
ON
  oi.product_id = p.id
WHERE
    oi.status = 'Complete'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY oi.user_id;

IndentationError: unexpected indent (<ipython-input-1-eeaf7e0f4ec8>, line 2)

Side project 02 - supply chain management
1. data governance
2. warehouse managment

In [ ]:
-- Domestic delivery data
SELECT
    oi.id AS item_id,
    u.postal_code AS customer_postal_code,
    u.country,
    u.state,
    u.city,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country = 'United States' -- US delievery

In [ ]:
-- International delivery data

SELECT
    oi.id AS item_id,
    u.postal_code AS customer_postal_code,
    u.country,
    u.state,
    u.city,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country != 'United States' -- Not US delievery

In [ ]:
-- Domestic delivery overall performance

WITH supply_chain_df AS
(SELECT
    oi.id AS item_id,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
    order_id,
    u.postal_code AS customer_postal_code,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country = 'United States') -- US delievery

SELECT
    COUNT(item_id) AS delivery_num,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, created_at, HOUR)),2) AS avg_lead_time,
    ROUND(AVG(TIMESTAMP_DIFF(shipped_at, created_at, HOUR)),2) AS avg_prep_time,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, shipped_at, HOUR)),2) AS avg_shipping_duration,
    ROUND(AVG(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        )
    ),2) AS avg_dis_to_customer,
    ROUND(SUM(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        ))/ (SUM(TIMESTAMP_DIFF(delivered_at, created_at, HOUR))),2)
    AS avg_delivery_rate
FROM supply_chain_df
ORDER BY delivery_num DESC, avg_lead_time ASC, avg_prep_time ASC, avg_shipping_duration ASC;

-- International delivery overall performance
WITH supply_chain_df AS
(SELECT
    oi.id AS item_id,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
    order_id,
    u.postal_code AS customer_postal_code,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country != 'United States') -- international delievery

SELECT
    COUNT(item_id) AS delivery_num,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, created_at, HOUR)),2) AS avg_lead_time,
    ROUND(AVG(TIMESTAMP_DIFF(shipped_at, created_at, HOUR)),2) AS avg_prep_time,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, shipped_at, HOUR)),2) AS avg_shipping_duration,
    ROUND(AVG(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        )
    ),2) AS avg_dis_to_customer,
    ROUND(SUM(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        ))/ (SUM(TIMESTAMP_DIFF(delivered_at, created_at, HOUR))),2)
    AS avg_delivery_rate
FROM supply_chain_df
ORDER BY delivery_num DESC, avg_lead_time ASC, avg_prep_time ASC, avg_shipping_duration ASC;

In [ ]:
-- Performance of each warehouse for domestic delivery

WITH supply_chain_df AS
(SELECT
    oi.id AS item_id,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
    order_id,
    u.postal_code AS customer_postal_code,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country = 'United States')

SELECT
    name,
    COUNT(item_id) AS delivery_num,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, created_at, HOUR)),2) AS avg_lead_time,
    ROUND(AVG(TIMESTAMP_DIFF(shipped_at, created_at, HOUR)),2) AS avg_prep_time,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, shipped_at, HOUR)),2) AS avg_shipping_duration,
    ROUND(AVG(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        )
    ),2) AS avg_dis_to_customer,
    ROUND(SUM(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        ))/ (SUM(TIMESTAMP_DIFF(delivered_at, created_at, HOUR))),2)
    AS avg_delivery_rate
FROM supply_chain_df
GROUP BY name
ORDER BY delivery_num DESC, avg_lead_time ASC, avg_prep_time ASC, avg_shipping_duration ASC;

-- Performance of each warehouse for international delivery

WITH supply_chain_df AS
(SELECT
    oi.id AS item_id,
    oi.delivered_at,
    oi.shipped_at,
    oi.created_at,
    order_id,
    u.postal_code AS customer_postal_code,
    u.longitude AS customer_longitude,
    u.latitude AS customer_latitude,
    ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
    dc.name,
    dc.longitude AS dc_longitude,
    dc.latitude AS dc_latitude
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON
    oi.inventory_item_id = ii.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON
    ii.product_distribution_center_id = dc.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
    AND oi.status = 'Complete'
    AND u.country != 'United States')

SELECT
    name,
    COUNT(item_id) AS delivery_num,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, created_at, HOUR)),2) AS avg_lead_time,
    ROUND(AVG(TIMESTAMP_DIFF(shipped_at, created_at, HOUR)),2) AS avg_prep_time,
    ROUND(AVG(TIMESTAMP_DIFF(delivered_at, shipped_at, HOUR)),2) AS avg_shipping_duration,
    ROUND(AVG(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        )
    ),2) AS avg_dis_to_customer,
    ROUND(SUM(
        ST_DISTANCE(
            ST_GEOGPOINT(customer_longitude, customer_latitude),
            ST_GEOGPOINT(dc_longitude, dc_latitude)
        ))/ (SUM(TIMESTAMP_DIFF(delivered_at, created_at, HOUR))),2)
    AS avg_delivery_rate
FROM supply_chain_df
GROUP BY name
ORDER BY delivery_num DESC, avg_lead_time ASC, avg_prep_time ASC, avg_shipping_duration ASC;

Side project 03 - sales predictions / time-series analysis

In [ ]:
-- theLook historical sales trend

SELECT
    year,
    month,
    CONCAT(CAST(year AS STRING), '-', CAST(month AS STRING)) AS time_period,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    (
        SELECT *,
            EXTRACT(YEAR FROM TIMESTAMP(created_at)) AS year,
            EXTRACT(MONTH FROM TIMESTAMP(created_at)) AS month
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
    ) AS o
LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
     EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) BETWEEN 2019 AND 2024
     AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) BETWEEN 1 AND 12
     AND oi.status NOT IN ('Cancelled', 'Returned')
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;